In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")

# print("Path to model files:", path)

In [6]:
import utils as u
import tensorflow_hub as hub

yamnet = hub.load('/Users/zhaokaiyang/.cache/kagglehub/models/google/yamnet/tensorFlow2/yamnet/1')

In [3]:
# 指定你的音频文件夹路径
folder_path = 'DEAM_Dataset/MEMD_audio/'
# 处理文件夹并获取 embeddings
embeddings_dict = u.process_folder(folder_path, yamnet)

2024-04-27 22:32:45.158159: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-27 22:32:45.166529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [4]:
# 打印所有键
print(list(embeddings_dict.keys()))

['746.mp3', '1588.mp3', '1563.mp3', '1205.mp3', '791.mp3', '949.mp3', '1211.mp3', '1577.mp3', '2056.mp3', '1239.mp3', '961.mp3', '2042.mp3', '550.mp3', '236.mp3', '222.mp3', '544.mp3', '1946.mp3', '1952.mp3', '1007.mp3', '1761.mp3', '1775.mp3', '1013.mp3', '1985.mp3', '1749.mp3', '1991.mp3', '342.mp3', '356.mp3', '430.mp3', '1832.mp3', '1826.mp3', '1198.mp3', '1173.mp3', '381.mp3', '1615.mp3', '1601.mp3', '395.mp3', '1167.mp3', '1629.mp3', '140.mp3', '632.mp3', '154.mp3', '82.mp3', '96.mp3', '168.mp3', '1417.mp3', '1371.mp3', '829.mp3', '1365.mp3', '69.mp3', '197.mp3', '1403.mp3', '801.mp3', '41.mp3', '815.mp3', '1359.mp3', '55.mp3', '7.mp3', '54.mp3', '1358.mp3', '814.mp3', '40.mp3', '800.mp3', '68.mp3', '1364.mp3', '1402.mp3', '196.mp3', '182.mp3', '1416.mp3', '1370.mp3', '169.mp3', '83.mp3', '155.mp3', '1628.mp3', '1600.mp3', '1166.mp3', '1172.mp3', '1614.mp3', '380.mp3', '1827.mp3', '1199.mp3', '419.mp3', '1833.mp3', '357.mp3', '431.mp3', '425.mp3', '343.mp3', '1990.mp3', '1748.mp3

In [5]:
# 创建一个新字典，其键是转换后的格式
new_embeddings_dict = {}
for key in embeddings_dict.keys():
    new_key = key[:-4] + ".0"  # 移除 '.mp3' 并添加 '.0'
    new_embeddings_dict[new_key] = embeddings_dict[key]

embeddings_dict = new_embeddings_dict


In [6]:
# 打印所有键
print(list(embeddings_dict.keys()))

['746.0', '1588.0', '1563.0', '1205.0', '791.0', '949.0', '1211.0', '1577.0', '2056.0', '1239.0', '961.0', '2042.0', '550.0', '236.0', '222.0', '544.0', '1946.0', '1952.0', '1007.0', '1761.0', '1775.0', '1013.0', '1985.0', '1749.0', '1991.0', '342.0', '356.0', '430.0', '1832.0', '1826.0', '1198.0', '1173.0', '381.0', '1615.0', '1601.0', '395.0', '1167.0', '1629.0', '140.0', '632.0', '154.0', '82.0', '96.0', '168.0', '1417.0', '1371.0', '829.0', '1365.0', '69.0', '197.0', '1403.0', '801.0', '41.0', '815.0', '1359.0', '55.0', '7.0', '54.0', '1358.0', '814.0', '40.0', '800.0', '68.0', '1364.0', '1402.0', '196.0', '182.0', '1416.0', '1370.0', '169.0', '83.0', '155.0', '1628.0', '1600.0', '1166.0', '1172.0', '1614.0', '380.0', '1827.0', '1199.0', '419.0', '1833.0', '357.0', '431.0', '425.0', '343.0', '1990.0', '1748.0', '1984.0', '1774.0', '586.0', '1012.0', '1006.0', '592.0', '1760.0', '1953.0', '579.0', '1947.0', '223.0', '551.0', '237.0', '960.0', '2043.0', '2057.0', '974.0', '1238.0', '

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 加载 Arousal & Valence 数据
av_data = pd.read_csv('DEAM_Dataset/annotations/static_annotations_averaged_songs_1_2000.csv')  # 确保路径正确
print(av_data.head())

   song_id   valence_mean   valence_std   arousal_mean   arousal_std
0        2            3.1          0.94            3.0          0.63
1        3            3.5          1.75            3.3          1.62
2        4            5.7          1.42            5.5          1.63
3        5            4.4          2.01            5.3          1.85
4        7            5.8          1.47            6.4          1.69


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 加载 Arousal & Valence 数据
av_data1 = pd.read_csv('DEAM_Dataset/annotations/static_annotations_averaged_songs_1_2000.csv')  # 确保路径正确
av_data2 = pd.read_csv('DEAM_Dataset/annotations/static_annotations_averaged_songs_2000_2058.csv')
av_data = pd.concat([av_data1, av_data2], ignore_index=True)
av_data.columns = av_data.columns.str.strip()  # 清理列名

# 初始化数据列表
X = []
y = []

# 遍历数据行
for index, row in av_data.iterrows():
    music_id = str(row['song_id'])  # 根据需要将 musicId 转换为字符串，假设 embeddings_dict 的键是字符串形式的 musicId
    if music_id in embeddings_dict:
        X.append(embeddings_dict[music_id])
        y.append([row['valence_mean'], row['arousal_mean']])
    else:
        print(f"Music ID {music_id} not found in embeddings_dict")  # 输出未找到的音乐 ID

# 将列表转换为 NumPy 数组
X = np.array(X)
y = np.array(y)

print(f"Total samples loaded: {len(X)}")  # 输出加载的样本总数

if len(X) > 0:
    # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
else:
    print("No data to split. Check your dataset and embeddings.")


Total samples loaded: 1802
Training set size: 1441, Test set size: 361


In [10]:
from models import build_model

# 根据输入数据的形状创建模型
model = build_model(input_shape=(60, 1024))  # 确保这与你的 embeddings 形状匹配



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

In [13]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-04-27 23:03:15.506759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


46/46 [==============================] - ETA: 0s - loss: 2.8258 - mae: 1.1994WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x177010670> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-04-27 23:03:22.413601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


46/46 [==============================] - 9s 145ms/step - loss: 2.8258 - mae: 1.1994 - val_loss: 0.9677 - val_mae: 0.7851
Epoch 2/20
46/46 [==============================] - 3s 65ms/step - loss: 0.8059 - mae: 0.7145 - val_loss: 1.0215 - val_mae: 0.7947
Epoch 3/20
46/46 [==============================] - 3s 66ms/step - loss: 0.7740 - mae: 0.6942 - val_loss: 0.8128 - val_mae: 0.7083
Epoch 4/20
46/46 [==============================] - 3s 65ms/step - loss: 0.6231 - mae: 0.6162 - val_loss: 1.0000 - val_mae: 0.8035
Epoch 5/20
46/46 [==============================] - 3s 64ms/step - loss: 0.8074 - mae: 0.7040 - val_loss: 0.8099 - val_mae: 0.7222
Epoch 6/20
46/46 [==============================] - 3s 63ms/step - loss: 0.4594 - mae: 0.5375 - val_loss: 0.8602 - val_mae: 0.7361
Epoch 7/20
46/46 [==============================] - 3s 64ms/step - loss: 0.4027 - mae: 0.4926 - val_loss: 1.1157 - val_mae: 0.8432
Epoch 8/20
46/46 [==============================] - 3s 63ms/step - loss: 0.5751 - mae: 0.6061

In [14]:
# 评估模型性能
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


12/12 [==============================] - 0s 24ms/step - loss: 0.8819 - mae: 0.7486
Test Loss: 0.8818883299827576, Test MAE: 0.7485961318016052


In [15]:
model.save('/Users/zhaokaiyang/Documents/24Spring/Deep Learning/Final Project/emotionPredict.h5')  # 保存为 SavedModel 格式

In [1]:
from tensorflow.keras.models import load_model
from tcn import TCN  # 确保正确导入 TCN 层的定义

# 加载模型时指定自定义层
model_test = load_model('/Users/zhaokaiyang/Documents/24Spring/Deep Learning/Final Project/emotionPredict.h5', custom_objects={'TCN': TCN})

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report 

2024-04-27 23:23:53.465091: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-27 23:23:53.465180: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
import librosa

wave, sr = librosa.load("DEAM_Dataset/MEMD_audio/7.mp3", sr=16000, mono=True)

In [12]:
import numpy as np

feature_test = u.extract_features(wave, yamnet)
feature_test = np.expand_dims(feature_test, axis=0)

In [8]:
feature_test.shape

(1, 60, 1024)

In [13]:
predicted_values = model_test.predict(feature_test)

1/1 [==============================] - 0s 30ms/step


In [14]:
arousal_test, valence_test = predicted_values[0][0], predicted_values[0][1]

In [15]:
print(arousal_test, valence_test)

5.594452 6.169183
